In [ ]:
#  Copyright (c) Microsoft Corporation.
#  Licensed under the MIT License.

In [ ]:
import sys, site
from pathlib import Path

################################# NOTE #################################
#  Please be aware that if colab installs the latest numpy and pyqlib  #
#  in this cell, users should RESTART the runtime in order to run the  #
#  following cells successfully.                                       #
########################################################################

try:
    import qlib
except ImportError:
    # install qlib
    ! pip install --upgrade numpy
    ! pip install pyqlib
    # reload
    site.main()

scripts_dir = Path.cwd().parent.joinpath("scripts")
if not scripts_dir.joinpath("get_data.py").exists():
    # download get_data.py script
    scripts_dir = Path("~/tmp/qlib_code/scripts").expanduser().resolve()
    scripts_dir.mkdir(parents=True, exist_ok=True)
    import requests
    with requests.get("https://raw.githubusercontent.com/microsoft/qlib/main/scripts/get_data.py") as resp:
        with open(scripts_dir.joinpath("get_data.py"), "wb") as fp:
            fp.write(resp.content)

In [ ]:

import pandas as pd
from qlib.config import REG_CN
from qlib.utils import exists_qlib_data, init_instance_by_config, flatten_dict
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.tests.data import GetData

In [ ]:
# use default data
provider_uri = "~/.qlib/qlib_data/cn_data"  # target_dir
if not exists_qlib_data(provider_uri):
    print(f"Qlib data is not found in {provider_uri}")
    GetData().qlib_data(target_dir=provider_uri, region=REG_CN)

qlib.init(provider_uri=provider_uri, region=REG_CN)

In [ ]:
market = "csi300"
benchmark = "SH000300"

###################################
# train model
###################################

data_handler_config = {
    "start_time": "2008-01-01",
    "end_time": "2020-08-01",
    "fit_start_time": "2008-01-01",
    "fit_end_time": "2014-12-31",
    "instruments": market,
}

task = {
    "model": {
        "class": "LGBModel",
        "module_path": "qlib.contrib.model.gbdt",
        "kwargs": {
            "loss": "mse",
            "colsample_bytree": 0.8879,
            "learning_rate": 0.0421,
            "subsample": 0.8789,
            "lambda_l1": 205.6999,
            "lambda_l2": 580.9768,
            "max_depth": 8,
            "num_leaves": 210,
            "num_threads": 20,
        },
    },
    "dataset": {
        "class": "DatasetH",
        "module_path": "qlib.data.dataset",
        "kwargs": {
            "handler": {
                "class": "Alpha158",
                "module_path": "qlib.contrib.data.handler",
                "kwargs": data_handler_config,
            },
            "segments": {
                "train": ("2008-01-01", "2014-12-31"),
                "valid": ("2015-01-01", "2016-12-31"),
                "test": ("2017-01-01", "2020-08-01"),
            },
        },
    },
}
# model initialization
model = init_instance_by_config(task["model"])
dataset = init_instance_by_config(task["dataset"])

# start exp to train model
with R.start(experiment_name="train_model"):
    R.log_params(**flatten_dict(task))
    model.fit(dataset)
    R.save_objects(trained_model=model)
    rid = R.get_recorder().id


In [ ]:
trade_start_time = "2017-01-01"
trade_end_time = "2020-08-01"

port_analysis_config = {
    "strategy": {
        "class": "TopkDropoutStrategy",
        "module_path": "qlib.contrib.strategy.model_strategy",
        "kwargs": {
            "step_bar": "week",
            "model": model,
            "dataset": dataset,
            "topk": 50,
            "n_drop": 5,
        },
    },
    "env": {
        "class": "SplitExecutor",
        "module_path": "qlib.contrib.backtest.executor",
        "kwargs": {
            "step_bar": "week",
            "generate_report": True,
            "sub_env": {
                "class": "SimulatorExecutor",
                "module_path": "qlib.contrib.backtest.executor",
                "kwargs": {
                    "step_bar": "day",
                    "verbose": True,
                    "generate_report": True,
                },
            },
            "sub_strategy": {
                "class": "SBBStrategyEMA",
                "module_path": "qlib.contrib.strategy.rule_strategy",
                "kwargs": {
                    "step_bar": "day",
                    "freq": "day",
                    "instruments": market,
                },
            },
        },
    },
    "backtest": {
        "start_time": trade_start_time,
        "end_time": trade_end_time,
        "account": 100000000,
        "benchmark": benchmark,
        "exchange_kwargs": {
            "freq": "day",
            "limit_threshold": 0.095,
            "deal_price": "close",
            "open_cost": 0.0005,
            "close_cost": 0.0015,
            "min_cost": 5,
        },
    },
}
# backtest and analysis
with R.start(experiment_name="backtest_analysis"):
    # prediction
    recorder = R.get_recorder()
    ba_rid = recorder.id
    sr = SignalRecord(model, dataset, recorder)
    sr.generate()

    # backtest & analysis
    par = PortAnaRecord(recorder, port_analysis_config, "day")
    par.generate()

In [ ]:
from qlib.contrib.report import analysis_model, analysis_position
from qlib.data import D
recorder = R.get_recorder(ba_rid, experiment_name="backtest_analysis")
pred_df = recorder.load_object("pred.pkl")
pred_df_dates = pred_df.index.get_level_values(level='datetime')
report_normal_df_1d = recorder.load_object("portfolio_analysis/report_normal_1day.pkl")
positions_1d = recorder.load_object("portfolio_analysis/positions_normal_1day.pkl")
analysis_df_1d = recorder.load_object("portfolio_analysis/port_analysis_1day.pkl")
report_normal_df_1w = recorder.load_object("portfolio_analysis/report_normal_1week.pkl")
positions_1w = recorder.load_object("portfolio_analysis/positions_normal_1week.pkl")
analysis_df_1w = recorder.load_object("portfolio_analysis/port_analysis_1week.pkl")

In [ ]:
analysis_position.report_graph(report_normal_df_1d)


In [ ]:
analysis_position.report_graph(report_normal_df_1w)

In [ ]:
analysis_position.risk_analysis_graph(analysis_df_1d, report_normal_df_1d)

In [ ]:
analysis_position.risk_analysis_graph(analysis_df_1w, report_normal_df_1w)